# Most salient topic for enwiki pages

The following transforms data from the snapshot of the page table for enwiki, page and talk page dumps, and a number of other MariaDB tables, then merges that stuff together with drafttopic API topic predictions for the corresponding latest revision IDs from the page table. It produces a "most salient" topic guess after applying a number of heuristics. Topic names are generally the drafttopic API names for topics, but there are also adjustments in a few places, most notably if an article has strong hints of being a settlement within a country, then the country name will be used.

See https://dr0ptp4kt.github.io/topics-8.html for an example of the first 10K page IDs (non-randomized) of the resultant output. As of this notebook you can also find the full resultant TSV dataset on notebook1003:~dr0ptp4kt/venv/topic_predictions_201912_mediawiki_page_dump_enriched_20191201_through_20191219.tsv.gz.


In a stream based context, one might react to recent changes events and then issue an Action API query something like along the following lines - https://en.wikipedia.org/w/api.php?action=query&format=json&prop=pageassessments%7Ccoordinates%7Ccategories%7Ctemplates%7Crevisions&continue=&titles=Stockholm&palimit=500&coprop=globe%7Ccountry&coprimary=primary&clshow=&cllimit=500&clcategories=&tllimit=500&tltemplates=&rvprop=content&rvslots=main - and then perform a number of the techniques in this notebook and referenced scripts. The script referenced in this notebook that goes through namespace 1 to find wikiprojects might be able to be replaced to a degree by querying the PageAssessments derived table: https://gerrit.wikimedia.org/r/plugins/gitiles/mediawiki/extensions/PageAssessments/+/master/db/addReviewsTable.sql, although an audit of the PageAssessments code might be helpful to look for edge cases and insert ordering.


## First things first, we hit the page table

In [1]:
%load_ext sql_magic

In [2]:
import findspark, os
os.environ['SPARK_HOME'] = '/usr/lib/spark2';
findspark.init()
import pyspark
import pyspark.sql
conf = pyspark.SparkConf().setMaster("yarn")  # Use master yarn here if you are going to query large datasets.
conf.set('spark.executor.memory', '8g')
conf.set('spark.yarn.executor.memoryOverhead', '1024')
conf.set('spark.executor.cores', '4')
conf.set('spark.dynamicAllocation.maxExecutors', '32')
conf.set('spark.driver.memory', '4g')
conf.set('spark.driver.maxResultSize', '10g')
conf.set('spark.logConf', True)
sc = pyspark.SparkContext(conf=conf)
spark_hive = pyspark.sql.HiveContext(sc)

%config SQL.conn_name = 'spark_hive'

In [3]:
%reload_ext sql

In [4]:
%%bash
analytics-mysql enwiki --print-target

dbstore1003.eqiad.wmnet:3311


In [5]:
%sql mysql+pymysql://dbstore1003.eqiad.wmnet:3311/enwiki?read_default_file=/etc/mysql/conf.d/research-client.cnf

'Connected: @enwiki'

In [6]:
%sql describe text

 * mysql+pymysql://dbstore1003.eqiad.wmnet:3311/enwiki?read_default_file=/etc/mysql/conf.d/research-client.cnf
3 rows affected.


Field,Type,Null,Key,Default,Extra
old_id,int(8) unsigned,NO,PRI,None,auto_increment
old_text,mediumblob,NO,,None,
old_flags,tinyblob,NO,,None,


In [12]:
%%read_sql enwiki_pv_rev -d

select page_id, page_title, page_latest as rev_id
from wmf_raw.mediawiki_page
where wiki_db = 'enwiki'
and snapshot = '2019-11'
and page_id is not null
and page_namespace = 0
and not page_is_redirect

Query started at 09:05:07 PM UTC; Query executed in 1.13 m

In [15]:
enwiki_pv_rev.shape

(5976292, 3)

In [14]:
enwiki_pv_rev[enwiki_pv_rev['page_title'] == 'United_States']

,page_id,page_title,rev_id
782964,3434750,United_States,928636548


## Now we try to find indicators of pages bearing geocoordinates or being about people

In [17]:
result2 = %sql select p.page_id, p.page_title, p.page_latest, c.is_human, g.has_geo from (select page_id, page_title, page_latest from page where page_id is not null and page_namespace = 0 and not page_is_redirect) as p left join (select distinct cl_from, true as is_human from categorylinks where cl_to rlike '\\d+.*[Dd]eaths' or cl_to in('Living_people', 'Year_of_death_missing', 'Year_of_death_unknown', 'Possibly_living_people', 'Missing_people')) as c on p.page_id = c.cl_from left join (select distinct gt_page_id, true as has_geo from geo_tags) as g on p.page_id = g.gt_page_id

 * mysql+pymysql://dbstore1003.eqiad.wmnet:3311/enwiki?read_default_file=/etc/mysql/conf.d/research-client.cnf
5981041 rows affected.


In [23]:
cat_df = result2.DataFrame()

In [26]:
cat_df['page_title'] = cat_df['page_title'].apply(lambda x: x.decode("utf-8"))

## Then we dump the page table's revision IDs for each page to a file and ask ORES for drafttopic predictions

In [30]:
enwiki_pv_rev[['rev_id']].dropna().astype('int64').to_json(path_or_buf='input_rev_id.json', orient='records', lines=True)

In [ ]:
%%bash
ores score_revisions https://ores.wikimedia.org "<email> analyzing article topics - ring twice at <number> if need be" enwiki drafttopic --parallel-requests=4 --input=input_rev_id.json > output_drafttopic_enwiki.json

## Uh oh, flappage. The next cells are just dealing with the flappage.

In [32]:
%%bash
wc -l output_drafttopic_enwiki.json

1756198 output_drafttopic_enwiki.json


In [38]:
%%bash
tail -5 output_drafttopic_enwiki.json | jq '.rev_id'

886234089
925573175
853359819
905556909


parse error: Unfinished JSON term at EOF at line 5, column 985


In [34]:
%%bash
tail -1 output_drafttopic_enwiki.json

{"score": {"drafttopic": {"score": {"probability": {"History_And_Society.Education": 0.0005988204653033881, "Culture.Media": 0.0023547281080837255, "History_And_Society.History and society": 0.18288022517886934, "STEM.Chemistry": 0.0007219595275585911, "Geography.Landforms": 0.0020163942780201294, "STEM.Mathematics": 0.0002150908095397874, "Assistance.Files": 0.002767229421250237, "STEM.Information science": 0.0011310204931930586, "Culture.Entertainment": 0.005882964199871971, "STEM.Meteorology": 0.00378660350868564, "Culture.Sports": 0.0015302668570903776, "Assistance.Article improvement and grading": 0.00012121435899339624, "Culture.Internet culture": 0.0007672245824915986, "Geography.Oceania": 0.048912931696032266, "History_And_Society.Business and economics": 0.005577135551959895, "Culture.Visual arts": 0.002108777686312782, "STEM.Technology": 0.00296474223513237, "Culture.Performing arts": 0.0006419956515468394, "STEM.Biology": 0.9988488304757314, "STEM.Engineering"

In [39]:
%%bash
grep -n '886234089\|925573175\|853359819\|905556909' input_rev_id.json

1756195:{"rev_id":886234089}
1756196:{"rev_id":925573175}
1756197:{"rev_id":853359819}
1756198:{"rev_id":905556909}


In [46]:
%%bash
wc -l input_rev_id.json

5976291 input_rev_id.json


In [56]:
%%bash
tail -4220100 input_rev_id.json | head -10
echo "okay step forward"
tail -4220094 input_rev_id.json | head -10



{"rev_id":917643631}
{"rev_id":925573198}
{"rev_id":886234089}
{"rev_id":925573175}
{"rev_id":853359819}
{"rev_id":905556909}
{"rev_id":904306868}
{"rev_id":897252693}
{"rev_id":883890176}
{"rev_id":833234732}
okay step forward
{"rev_id":904306868}
{"rev_id":897252693}
{"rev_id":883890176}
{"rev_id":833234732}
{"rev_id":867772546}
{"rev_id":866877593}
{"rev_id":923153907}
{"rev_id":917100789}
{"rev_id":847142980}
{"rev_id":880458064}


In [57]:
%%bash
tail -4220094 input_rev_id.json > input_rev_id_resuming.json

In [58]:
%%bash
head -1756197 output_drafttopic_enwiki.json > output_drafttopic_enwiki_trimmed.json
mv output_drafttopic_enwiki_trimmed.json output_drafttopic_enwiki.json

In [60]:
%%bash
tail -5 output_drafttopic_enwiki.json | jq '.rev_id'

917643631
925573198
886234089
925573175
853359819


In [ ]:
%%bash
ores score_revisions https://ores.wikimedia.org "<email> analyzing article topics - ring twice at <number> if need be" enwiki drafttopic --parallel-requests=4 --input=input_rev_id_resuming.json > output_drafttopic_enwiki_resuming.json

In [62]:
%%bash
wc -l output_drafttopic_enwiki_resuming.json

3146546 output_drafttopic_enwiki_resuming.json


In [63]:
%%bash
tail -5 output_drafttopic_enwiki_resuming.json | jq '.rev_id'

916329225
902875015
725824139
761985934


parse error: Unfinished JSON term at EOF at line 5, column 582


In [64]:
%%bash
tail -1 output_drafttopic_enwiki_resuming.json

{"score": {"drafttopic": {"score": {"probability": {"STEM.Chemistry": 0.0005167404666004942, "Assistance.Files": 0.00033546774995569275, "STEM.Information science": 0.0011145079148051893, "Geography.Oceania": 0.015924831752445974, "Culture.Performing arts": 0.0005097292528512669, "History_And_Society.Transportation": 0.0010427643924574323, "Culture.Philosophy and religion": 0.005539325569185675, "STEM.Biology": 0.0017130552344170537, "Culture.Crafts and hobbies": 0.0007668392384482391, "Assistance.Maintenance": 0.01502344426856681, "Assistance.Article improvement and grading"

In [65]:
%%bash
grep -n '916329225\|902875015\|725824139\|761985934' input_rev_id.json

4902741:{"rev_id":916329225}
4902742:{"rev_id":902875015}
4902743:{"rev_id":725824139}
4902744:{"rev_id":761985934}


In [70]:
%%bash
tail -1073552 input_rev_id.json | head -10
echo "okay step forward"
tail -1073548 input_rev_id.json | head -10

{"rev_id":916329225}
{"rev_id":902875015}
{"rev_id":725824139}
{"rev_id":761985934}
{"rev_id":875794201}
{"rev_id":883250759}
{"rev_id":879260374}
{"rev_id":840577794}
{"rev_id":909829877}
{"rev_id":927032359}
okay step forward
{"rev_id":875794201}
{"rev_id":883250759}
{"rev_id":879260374}
{"rev_id":840577794}
{"rev_id":909829877}
{"rev_id":927032359}
{"rev_id":906625072}
{"rev_id":922734317}
{"rev_id":914654246}
{"rev_id":688759624}


In [71]:
%%bash
tail -1073548 input_rev_id.json > input_rev_id_resuming_hopefully_final.json

In [72]:
%%bash
head -3146545 output_drafttopic_enwiki_resuming.json > output_drafttopic_enwiki_trimmed_resuming.json
mv output_drafttopic_enwiki_trimmed_resuming.json output_drafttopic_enwiki_resuming.json

In [73]:
%%bash
tail -5 output_drafttopic_enwiki_resuming.json | jq '.rev_id'

921781541
853956599
916329225
902875015
725824139


In [74]:
%%bash
head -2 input_rev_id_resuming_hopefully_final.json

{"rev_id":875794201}
{"rev_id":883250759}


In [75]:
# oops, need to insert a row to query against ores - {"rev_id":761985934}, just did that manually

In [81]:
%%bash
wc -l input_rev_id_resuming_hopefully_final.json
head -3 input_rev_id_resuming_hopefully_final.json

1073549 input_rev_id_resuming_hopefully_final.json
{"rev_id":761985934}
{"rev_id":875794201}
{"rev_id":883250759}


In [ ]:
%%bash
ores score_revisions https://ores.wikimedia.org "<email> analyzing article topics - ring twice at <number> if need be" enwiki drafttopic --parallel-requests=4 --input=input_rev_id_resuming_hopefully_final.json > output_drafttopic_enwiki_resuming_hopefully_final.json

In [83]:
%%bash
wc -l output_drafttopic_enwiki_resuming_hopefully_final.json

1073549 output_drafttopic_enwiki_resuming_hopefully_final.json


In [86]:
%%bash
tail -5 output_drafttopic_enwiki_resuming_hopefully_final.json | jq '.rev_id'

928683645
928683693
928684152
928685685
928685777


In [87]:
%%bash
wc -l output_drafttopic_enwiki_resuming.json
wc -l output_drafttopic_enwiki.json

3146545 output_drafttopic_enwiki_resuming.json
1756197 output_drafttopic_enwiki.json


In [88]:
%%bash
cat output_drafttopic_enwiki_resuming.json >> output_drafttopic_enwiki.json
cat output_drafttopic_enwiki_resuming_hopefully_final.json >> output_drafttopic_enwiki.json

In [89]:
%%bash
wc -l output_drafttopic_enwiki.json

5976291 output_drafttopic_enwiki.json


## Okay, we have as many of the drafttopic predictions as possible. Note there were some bad rows, often deleted revisions but also a few non-responsive API things. But there aren't enough to probably be worried about it for practical purposes.

In [90]:
%%bash
wc -l ../topicmodeling/ns1.5.tsv

7883967 ../topicmodeling/ns1.5.tsv


## Wait what!? Where did this ns1.5.tsv file come from?

That file is produced by running
`python parse_text_dumps_ns1.5.py > ns1.5.tsv`
from the ../topicmodeling directory. parse_text_dumps_ns1.5.py just goes through all of the Talk pages on enwiki and tries to extract WikiProject template invocations and then puts them into a file.

You'll notice in parse_text_dumps_ns1.5.py a reference to ```open('wikiproject_redirects_ns1.5.tsv')```. That TSV is built with the following Hive query:

```select page_title, rd_title, rd_namespace from mediawiki_page inner join mediawiki_redirect on (mediawiki_page.page_id = mediawiki_redirect.rd_from and mediawiki_page.wiki_db = 'enwiki' and mediawiki_page.snapshot = '2019-11' and mediawiki_page.page_namespace = 10 and mediawiki_redirect.rd_title rlike '(?i)^wikiproject_' and mediawiki_redirect.rd_title not rlike '(?i)(\/doc|\/sandbox|\/class|\/test|\/assessment|\/testcases|\/to_do|\/todo|\/navbox|\/nav|\/category_list)$' and mediawiki_redirect.rd_title not rlike '(?i)\/userbox' and mediawiki_redirect.wiki_db = 'enwiki' and mediawiki_redirect.snapshot = '2019-11');```

## Next, we parse the ns1.5.tsv file and try to predict the "most important" WikiProject.

The following just tries to find the wikiproject that had the highest importance rating based on the article's talk page. If more than one wikiproject had the same highest importance rating, the first one that showed up becomes the most important one. Additionally, the first wikiproject that showed up at all is captured; sometimes editors put what might be the most salient wikiproject at the top of the talk page, but without any importance rating! Importance is actually importance with respect to the given wikiproject's corpus, but it turns out to be a useful ranking criteria.

For fun, we output a listing of how many articles fell in each wikiproject.

In [92]:
import csv
import re
from collections import defaultdict, OrderedDict
import pandas as pd

importances = ['unknown_importance', 'low', 'mid', 'high', 'top']
mid_level_categories = defaultdict(int)
bio_counter = 0

title_topic_list = []

with open('../topicmodeling/ns1.5.tsv') as fin:
    tsv_reader = csv.reader(fin, delimiter='\t')
    counter = 0
    column_names = next(tsv_reader)
    for line in tsv_reader:
        highest_importance = -1
        first_valid_category_encountered = None
        best_category = None
        marked_as_bio = False
        if counter > 7883967:
            break
        else:
            title = re.sub(' ', '_', line[1])
            page_id = line[2]
            projects = line[12].split('!!!')
            for project in projects:
                try:
                    k, v = project.split('~~~')
                except Exception:
                    k = project
                    v = 'unknown_importance'
                # print(title + '\t' + k +'\t' + v.lower())
                if k not in ['WikiProject banner shell', 'no_wikiprojects']:
                    if not first_valid_category_encountered:
                        first_valid_category_encountered = k
                        first_valid_category_encountered_rating = v
                    if k == 'WikiProject Biography':
                        marked_as_bio = True
                        bio_counter +=1
                    try:
                        rating = importances.index(v)
                    except ValueError:
                        rating = -1
                    if rating > highest_importance:
                        highest_importance = rating
                        highest_importance_rating = v
                        best_category = k
                        # print("Best category so far: " + best_category)
            # print("Best category: "  + str(best_category))
            # print("Marked as biography: " + str(marked_as_bio) + "\n")
            mid_level_categories[str(best_category)] += 1
            title_topic_list.append([title, page_id, best_category, highest_importance_rating, first_valid_category_encountered, first_valid_category_encountered_rating])
            counter += 1
    print("Biographies encountered: " + str(bio_counter))
    for k,v in OrderedDict(sorted(mid_level_categories.items(), key=lambda t: t[1])).items():
        print(k + ": " + str(v))

    title_to_topic_df = pd.DataFrame(title_topic_list, columns=['title', 'page_id_ns_1', 'topic', 'topic_rating', 'topic_first_encountered', 'topic_first_encountered_rating'])



Biographies encountered: 1675097
WikiProject  Mixed martial arts: 1
WikiProject Texas Tech University: 1
WikiProject  Military history: 1
WikiProject  Malaysia: 1
WIkiProject Rwanda: 1
WikiProject  Iraq: 1
WikiProject  Brazil: 1
WikiProject Countering systemic bias: 1
WikiProject  Chile: 1
WikiProject New Zealand Railways: 1
WikiProject  Ukraine: 1
WikiProject  Hinduism: 1
WikiProject  Catholicism: 1
WikiProject  Puerto Rico: 1
WikiProject Pteridophytes: 1
WikiProject Rajasthan: 1
WikiProject  Hong Kong: 1
WikiProject  Armenia: 1
WikiProject  Albums: 1
WikiProject  Astronomy: 1
WikiProject Film Award: 1
WikiProject Infoboxes: 1
WikiProject Mathematical and Computational Biology: 1
WikiProject  New York: 1
WikiProject  Philippines: 1
WikiProject Table  tennis: 1
WikiProject  Meteorology: 1
WikiProject  GeorgiaUS: 1
WikiProject  Bulgaria: 1
WikiProject Nonviolence: 1
WikiProject  Linguistics: 1
WikiProject Utah: 1
WikIProject Belgium: 1
WikiProject Current events: 1
WikiProject  Belarus:

In [93]:
del title_topic_list

In [94]:
len(title_to_topic_df)

7883887

In [97]:
title_to_topic_df

,title,page_id_ns_1,topic,topic_rating,topic_first_encountered,topic_first_encountered_rating
0,Atlas_Shrugged,128,WikiProject Objectivism,top,WikiProject Objectivism,top
1,Algeria,354,WikiProject Africa,top,WikiProject Africa,top
2,Altruism/Archive_1,582,None,top,None,top
3,Arc_de_Triomphe,672,WikiProject France,top,WikiProject France,top
4,Archaeology,692,WikiProject Archaeology,top,WikiProject Archaeology,top
5,Android_(robot),714,WikiProject Robotics,high,WikiProject Robotics,high
6,Animal,722,WikiProject Animals,top,WikiProject Animals,top
7,American_football,774,WikiProject American football,top,WikiProject United States,high
8,Assistive_technology,793,WikiProject Disability,mid,WikiProject Accessibility,unknown_importance
9,Afghanistan/Archive_6,823,None,mid,None,unknown_importance


## Next, we parse the drafttopic predictions sorted on the top five highest probability predictions. Notice here we see some bad rows, as indicated before...that's probably okay, we don't keep them.

In [ ]:
import pandas as pd
import json


rev_to_predicted_topic = []

# Get topic from ORES draft topic output
def get_pred_topic_best(rev_id, input_json):
    try:
        topics = input_json['score']['drafttopic']['score']['probability']
        #print(topics)
        #best = sorted(topics, key=topics.get, reverse=True)[0]
        sorted_topics = sorted(topics, key=topics.get, reverse=True)
        #print(sorted_topics)
        best1 = sorted_topics[0]
        best1_score = topics[best1]
        best2 = sorted_topics[1]
        best2_score = topics[best2]
        best3 = sorted_topics[2]
        best3_score = topics[best3]
        best4 = sorted_topics[3]
        best4_score = topics[best4]
        best5 = sorted_topics[4]
        best5_score = topics[best5]
    except (IndexError, KeyError) as error:
        best = 'Unknown'
    return [rev_id, best1, best1_score, best2, best2_score, best3, best3_score, best4, best4_score, best5, best5_score]

with open('output_drafttopic_enwiki.json') as json_file:
    for line in json_file:
        try:
            ores_results = json.loads(line)
            #print(ores_results)
            #print(ores_results['rev_id'])
            #get_pred_topic_best(ores_results)
            rev_to_predicted_topic.append(get_pred_topic_best(ores_results['rev_id'], ores_results))
        except UnboundLocalError:
            #print(line)

topic_df = pd.DataFrame(rev_to_predicted_topic, columns=['rev_id','best1', 'best1_score', 'best2', 'best2_score', 'best3', 'best3_score', 'best4', 'best4_score', 'best5', 'best5_score'])

In [101]:
del rev_to_predicted_topic

In [102]:
topic_df.shape[0]

5972819

In [103]:
topic_df.iloc[55555]

rev_id                                          927335065
best1                                 Geography.Countries
best1_score                                      0.864697
best2                                Culture.Plastic arts
best2_score                                     0.0942508
best3                              Assistance.Maintenance
best3_score                                     0.0797609
best4             History_And_Society.History and society
best4_score                                     0.0709831
best5          History_And_Society.Business and economics
best5_score                                     0.0417863
Name: 55555, dtype: object

In [105]:
cat_df.shape

(5981041, 5)

## Now we merge the page tables stuff with the indicators of whether an article might be about a human or have geocoordinates.

In [106]:
consolidated = pd.merge(enwiki_pv_rev, cat_df, on='page_id', how='left')

In [107]:
consolidated

,page_id,page_title_x,rev_id,page_title_y,page_latest,is_human,has_geo
0,12,Anarchism,928659071,Anarchism,930305821.0,NaN,NaN
1,25,Autism,928602598,Autism,928602598.0,NaN,NaN
2,39,Albedo,926891749,Albedo,926891749.0,NaN,NaN
3,290,A,927107123,A,927107123.0,NaN,NaN
4,303,Alabama,928520502,Alabama,930263332.0,NaN,1.0
5,305,Achilles,928351599,Achilles,928951332.0,NaN,NaN
6,307,Abraham_Lincoln,928533917,Abraham_Lincoln,930094843.0,1.0,NaN
7,308,Aristotle,928197558,Aristotle,929227456.0,1.0,NaN
8,309,An_American_in_Paris,926138379,An_American_in_Paris,930213356.0,NaN,NaN
9,316,Academy_Award_for_Best_Production_Design,923578186,Academy_Award_for_Best_Production_Design,923578186.0,NaN,NaN


## And then we merge that together with the highest importance wikiproject & wikiproject first encountered from the talk page, if available. We do this based on the title, which is good enough, although in the future we'd probably want to have the page ID if it were available on both sides of the article page and talk page (that's maybe a TODO).

In [109]:
consolidated = pd.merge(consolidated, title_to_topic_df, left_on='page_title_x', right_on='title', how='left')

In [110]:
consolidated

,page_id,page_title_x,rev_id,page_title_y,page_latest,is_human,has_geo,title,page_id_ns_1,topic,topic_rating,topic_first_encountered,topic_first_encountered_rating
0,12,Anarchism,928659071,Anarchism,930305821.0,NaN,NaN,Anarchism,416541,WikiProject Alternative Views,top,WikiProject Alternative Views,top
1,25,Autism,928602598,Autism,928602598.0,NaN,NaN,Autism,44741,WikiProject Autism,top,WikiProject Medicine,high
2,39,Albedo,926891749,Albedo,926891749.0,NaN,NaN,Albedo,47435,WikiProject Astronomy,high,WikiProject Physics,low
3,290,A,927107123,A,927107123.0,NaN,NaN,A,2554,WikiProject Writing systems,mid,WikiProject Writing systems,mid
4,303,Alabama,928520502,Alabama,930263332.0,NaN,1.0,Alabama,1284,WikiProject Alabama,top,WikiProject Alabama,top
5,305,Achilles,928351599,Achilles,928951332.0,NaN,NaN,Achilles,543840,WikiProject Greece,high,WikiProject Biography,unknown_importance
6,307,Abraham_Lincoln,928533917,Abraham_Lincoln,930094843.0,1.0,NaN,Abraham_Lincoln,2301,WikiProject Illinois,top,WikiProject Biography,unknown_importance
7,308,Aristotle,928197558,Aristotle,929227456.0,1.0,NaN,Aristotle,18539950,WikiProject Philosophy,top,WikiProject Biography,unknown_importance
8,309,An_American_in_Paris,926138379,An_American_in_Paris,930213356.0,NaN,NaN,An_American_in_Paris,1229991,WikiProject Classical music,unknown_importance,WikiProject Classical music,unknown_importance
9,316,Academy_Award_for_Best_Production_Design,923578186,Academy_Award_for_Best_Production_Design,923578186.0,NaN,NaN,Academy_Award_for_Best_Production_Design,4292425,WikiProject Awards,mid,WikiProject Film,unknown_importance


## And then we merge that with the top five drafttopic topic predictions where available. Here we can join on the revision ID.

In [111]:
consolidated = pd.merge(consolidated, topic_df, on='rev_id', how='left')

In [112]:
consolidated

,page_id,page_title_x,rev_id,page_title_y,page_latest,is_human,has_geo,title,page_id_ns_1,topic,...,best1,best1_score,best2,best2_score,best3,best3_score,best4,best4_score,best5,best5_score
0,12,Anarchism,928659071,Anarchism,930305821.0,NaN,NaN,Anarchism,416541,WikiProject Alternative Views,...,Culture.Language and literature,0.573744,History_And_Society.History and society,0.564499,Culture.Philosophy and religion,0.442044,Geography.Europe,0.337839,History_And_Society.Politics and government,0.305153
1,25,Autism,928602598,Autism,928602598.0,NaN,NaN,Autism,44741,WikiProject Autism,...,STEM.Medicine,0.990639,History_And_Society.History and society,0.451306,STEM.Biology,0.215708,Culture.Language and literature,0.140741,Geography.Countries,0.062510
2,39,Albedo,926891749,Albedo,926891749.0,NaN,NaN,Albedo,47435,WikiProject Astronomy,...,STEM.Space,0.690529,STEM.Meteorology,0.677404,STEM.Physics,0.331766,STEM.Geosciences,0.178165,History_And_Society.History and society,0.103957
3,290,A,927107123,A,927107123.0,NaN,NaN,A,2554,WikiProject Writing systems,...,Culture.Language and literature,0.336972,Geography.Europe,0.201948,History_And_Society.History and society,0.155678,Geography.Countries,0.127005,Assistance.Maintenance,0.079002
4,303,Alabama,928520502,Alabama,930263332.0,NaN,1.0,Alabama,1284,WikiProject Alabama,...,Geography.Countries,0.809819,History_And_Society.History and society,0.171643,History_And_Society.Politics and government,0.133188,Assistance.Maintenance,0.064497,Culture.Language and literature,0.064399
5,305,Achilles,928351599,Achilles,928951332.0,NaN,NaN,Achilles,543840,WikiProject Greece,...,History_And_Society.History and society,0.690594,Culture.Language and literature,0.469709,Geography.Europe,0.451748,Culture.Philosophy and religion,0.342202,Culture.Visual arts,0.148593
6,307,Abraham_Lincoln,928533917,Abraham_Lincoln,930094843.0,1.0,NaN,Abraham_Lincoln,2301,WikiProject Illinois,...,Geography.Countries,0.789089,Culture.Language and literature,0.762656,History_And_Society.History and society,0.322005,History_And_Society.Military and warfare,0.287212,History_And_Society.Politics and government,0.184107
7,308,Aristotle,928197558,Aristotle,929227456.0,1.0,NaN,Aristotle,18539950,WikiProject Philosophy,...,Culture.Language and literature,0.743942,Culture.Philosophy and religion,0.603628,History_And_Society.History and society,0.595194,Geography.Europe,0.177480,STEM.Science,0.111327
8,309,An_American_in_Paris,926138379,An_American_in_Paris,930213356.0,NaN,NaN,An_American_in_Paris,1229991,WikiProject Classical music,...,Culture.Performing arts,0.969374,Culture.Language and literature,0.500320,Geography.Countries,0.188465,Geography.Europe,0.136918,History_And_Society.History and society,0.065426
9,316,Academy_Award_for_Best_Production_Design,923578186,Academy_Award_for_Best_Production_Design,923578186.0,NaN,NaN,Academy_Award_for_Best_Production_Design,4292425,WikiProject Awards,...,Culture.Entertainment,0.937438,Assistance.Maintenance,0.177690,Culture.Visual arts,0.148124,Culture.Language and literature,0.106589,Culture.Performing arts,0.093777


In [113]:
del topic_df
del title_to_topic_df
del enwiki_pv_rev
del result2
del cat_df

In [114]:
sample300 = consolidated.sample(n=300)

In [115]:
sample300

,page_id,page_title_x,rev_id,page_title_y,page_latest,is_human,has_geo,title,page_id_ns_1,topic,...,best1,best1_score,best2,best2_score,best3,best3_score,best4,best4_score,best5,best5_score
2535092,21125046,National_Register_of_Historic_Places_listings_...,928547446,National_Register_of_Historic_Places_listings_...,929994921.0,NaN,1.0,National_Register_of_Historic_Places_listings_...,21128175,WikiProject National Register of Historic Places,...,Geography.Countries,0.869377,Culture.Plastic arts,0.268390,Geography.Bodies of water,0.065165,Assistance.Maintenance,0.037315,Assistance.Contents systems,0.023555
4437083,42333287,Pro_Basketball_League_MVP,918331272,Pro_Basketball_League_MVP,918331272.0,NaN,NaN,Pro_Basketball_League_MVP,44175466,WikiProject Belgium,...,Culture.Sports,0.847678,Culture.Language and literature,0.239679,Geography.Countries,0.178502,Geography.Europe,0.138604,Assistance.Maintenance,0.075091
5801190,60162308,Eveless,923603738,Eveless,923603738.0,NaN,NaN,Eveless,60162316,WikiProject Horror,...,Culture.Entertainment,0.938708,Culture.Internet culture,0.484581,Culture.Language and literature,0.266871,Geography.Countries,0.186136,Culture.Visual arts,0.176062
4804843,47015250,Goran_Jovanović,696711709,Goran_Jovanović,696711709.0,NaN,NaN,Goran_Jovanović,47015251,WikiProject Disambiguation,...,Culture.Language and literature,0.497037,Assistance.Maintenance,0.461824,Geography.Europe,0.338860,Culture.Sports,0.175212,Geography.Countries,0.082063
938703,4535221,Nancy_Borgenicht,913747168,Nancy_Borgenicht,913747168.0,1.0,NaN,Nancy_Borgenicht,6501777,WikiProject Biography,...,Culture.Language and literature,0.837300,Geography.Countries,0.246651,Culture.Entertainment,0.230890,Culture.Broadcasting,0.043423,History_And_Society.Business and economics,0.041013
4549300,43544790,Central_Road_Research_Institute,916025952,Central_Road_Research_Institute,916025952.0,NaN,NaN,Central_Road_Research_Institute,44466691,WikiProject India,...,Geography.Countries,0.448912,History_And_Society.Transportation,0.420949,STEM.Engineering,0.181463,Geography.Europe,0.168834,History_And_Society.Business and economics,0.109915
5050851,50298724,CABC,878553043,CABC,878553043.0,NaN,NaN,CABC,52129040,WikiProject Disambiguation,...,Geography.Countries,0.411181,Geography.Europe,0.155433,Assistance.Maintenance,0.146546,Culture.Philosophy and religion,0.144036,Culture.Sports,0.099321
1035657,5338669,Jan_Stuyt,917271799,Jan_Stuyt,917271799.0,1.0,1.0,Jan_Stuyt,6730837,WikiProject Architecture,...,Geography.Europe,0.938682,Culture.Plastic arts,0.868806,Culture.Philosophy and religion,0.664597,Culture.Language and literature,0.134143,History_And_Society.History and society,0.056596
1568692,10738195,Grenache_blanc,918560713,Grenache_blanc,918560713.0,NaN,NaN,Grenache_blanc,11071578,WikiProject Wine,...,Culture.Food and drink,0.998356,Geography.Europe,0.338079,STEM.Biology,0.095228,History_And_Society.History and society,0.062452,Assistance.Maintenance,0.050302
1135604,6109288,"Christchurch,_Newport",922404029,"Christchurch,_Newport",922404029.0,NaN,1.0,"Christchurch,_Newport",21703623,WikiProject Wales,...,Geography.Europe,0.887733,Geography.Oceania,0.751430,Culture.Plastic arts,0.216110,Culture.Philosophy and religion,0.074958,History_And_Society.Military and warfare,0.050626


In [116]:
%%bash
grep '^en' ../topicmodeling/ns0.3.tsv | wc -l
grep -v '^en' ../topicmodeling/ns0.3.tsv | wc -l
grep '^en' ../topicmodeling/ns0.3.tsv | cut -f5 | sed 's/~~~/\t/g' | cut -f1 | sort | uniq -c | sort -nr

436961
225
 356989 Infobox settlement
  37439 Infobox French commune
  13302 Infobox German location
   8011 Infobox Italian comune
   4609 Infobox Settlement
   2968 Infobox U.S. county
   2825 Infobox Swiss town
   2631 Infobox Greek Dimos
   1608 Infobox Portuguese subdivision
   1471 Infobox French subdivision
   1085 Infobox city Japan
    666 Infobox frazione
    596 Infobox kommune
    556 Infobox Israel village
    377 Infobox New Zealand suburbs
    262 Infobox Israel municipality
    240 Infobox Cape Verde settlement
    228 Infobox Russian inhabited locality
    186 "Infobox settlement
    183 Infobox Syrian settlement
    172 Infobox German district
     91 Pad top italic
     88 Infobox Russian federal subject
     88 Infobox Finnish municipality
     70 Infobox U.S. metropolitan area
     53 Infobox U.S. state
     28 Athabasca County Council
     18 Infobox state assembly constituency
     15 Calgary City Council
     13 Infobox province or territory of Canada
     13 In

## WHAAAAAAAT!? Where did that ns0.3.tsv file come from?

It's produced by running ```python parse_text_dumps_infoboxes.py > ns0.3.tsv``` in ../topicmodeling/.

When you look at parse_text_dumps_infobox.py you'll notice that it has a line ```with open('infobox-settlement-titles.list') as fin```.

That .list file is produced by running ../topicmodeling/infobox-settlement-titles.sql against the enwiki MariaDB analytics replica.

The .list has a bunch of common template redirects for the 'Infobox settlement' infobox.

parse_text_dumps_infoboxes.py just goes through article pages looking for templates with 'Infobox settlement' infoboxes and tries to figure out if the page is a settlement with a clearly defined country or international state.

The output produced in the ns0.3.tsv is very rudimentary ane needs more postprocessing.

## Now we do further processing to try to identify the country / international state from articles bearing the Infobox settlement template (or redirects to it)

You'll note that where it was possible to map up an Infobox settlement redirect title to a country, that's done. You'll also see mappings of country codes and a few edge case country codes that people use in practice.

In [117]:
import csv
import re
from collections import defaultdict, OrderedDict
import pandas as pd
import mwparserfromhell
import re

settlement_countries = {
'Infobox U.S. county': 'United States',
'Infobox Portuguese subdivision': 'Portugal',
'Infobox French subdivision': 'France',
'Infobox Albanian settlement': 'Albania',
'Infobox Belgium municipality': 'Belgium',
'Infobox French commune': 'France',
'Infobox German location': 'Germany',
'Infobox Italian comune': 'Italy',
'Infobox Swiss town' : 'Switzerland',
'Infobox Greek Dimos': 'Greece',
'Infobox city Japan': 'Japan',
'Infobox French canton': 'France',
'Infobox frazione': 'Italy',
'Infobox kommune': 'Norway',
'Infobox Israel village': 'Israel',
'Infobox New Zealand suburbs': 'New Zealand',
'Infobox French arrondissement': 'France',
'Infobox Israel municipality': 'Israel',
'Infobox Cape Verde settlement': 'Cape Verde',
'Infobox Russian inhabited locality': 'Russia',
'Infobox Syrian settlement': 'Syria',
'Infobox German district': 'Germany',
'Infobox French communauté': 'France',
'Pad top italic': 'Ireland',
'Infobox Finnish municipality': 'Finland',
'Infobox Russian federal subject': 'Russia',
'Infobox U.S. metropolitan area': 'United States',
'Infobox U.S. state': 'United States',
'Athabasca County Council': 'Canada',
'Infobox state assembly constituency': 'India',
'Calgary City Council': 'Canada',
'Infobox province or territory of Canada': 'Canada',
'Yellowhead County Council': 'Canada',
'Edmonton City Council': 'Canada',
'Infobox settlement myd': 'Malaysia',
'Infobox Nepal municipality': 'Nepal',
'Grande Prairie County Council': 'Canada',
'Leduc County Council': 'Canada',
'Rocky View County Council': 'Canada',
'Lac Ste. Anne County Council': 'Canada',
'Foothills No 31 Council': 'Canada',
'Clearwater County Council (Alberta)': 'Canada',
'Wood Buffalo Municipal Council': 'Canada',
'Wheatland County Council (Alberta)': 'Canada',
'Strathcona County Council': 'Canada',
'Parkland County Council': 'Canada',
'Mackenzie County Council': 'Canada',
'Mountain View County Council': 'Canada',
'Infobox Nepal Rajya': 'Nepal',
'Infobox Indian Jurisdiction': 'India'
}

template_countries = {
'US': 'United States',
'United States,': 'United States',
'United States of America': 'United States',
'SAM': 'Samoa',
'NGR': 'Nigeria',
'BAH': 'The Bahamas',
'ROM': 'Romania',
'PAR': 'Paraguay',
'INA': 'Indonesia',
'HON': 'Honduras',
'NZ': 'New Zealand',
'Falkland Islands': 'United Kingdom',
'CRO': 'Croatia',
'SUI': 'Switzerland',
'PRC': 'China',
'UAE': 'United Arab Emirates',
'In Crimea': 'Crimea',
'LAT': 'Latvia',
'ABW': 'Aruba',
'AFG': 'Afghanistan',
'AGO': 'Angola',
'AIA': 'Anguilla',
'ALA': 'Åland Islands',
'ALB': 'Albania',
'AND': 'Andorra',
'ARE': 'United Arab Emirates',
'ARG': 'Argentina',
'ARM': 'Armenia',
'ASM': 'American Samoa',
'ATA': 'Antarctica',
'ATF': 'French Southern and Antarctic Lands',
'ATG': 'Antigua and Barbuda',
'AUS': 'Australia',
'AUT': 'Austria',
'AZE': 'Azerbaijan',
'BDI': 'Burundi',
'BEL': 'Belgium',
'BEN': 'Benin',
'BES': 'Caribbean Netherlands',
'BFA': 'Burkina Faso',
'BGD': 'Bangladesh',
'BGR': 'Bulgaria',
'BHR': 'Bahrain',
'BHS': 'The Bahamas',
'BIH': 'Bosnia and Herzegovina',
'BLM': 'Saint Barthélemy',
'BLR': 'Belarus',
'BLZ': 'Belize',
'BMU': 'Bermuda',
'BOL': 'Bolivia',
'BRA': 'Brazil',
'BRB': 'Barbados',
'BRN': 'Brunei',
'BTN': 'Bhutan',
'BVT': 'Bouvet Island',
'BWA': 'Botswana',
'CAF': 'Central African Republic',
'CAN': 'Canada',
'CCK': 'Cocos (Keeling) Islands',
'CHE': 'Switzerland',
'CHL': 'Chile',
'CHN': 'China',
'CIV': 'Ivory Coast',
'CMR': 'Cameroon',
'COD': 'Democratic Republic of the Congo',
'COG': 'Republic of the Congo',
'COK': 'Cook Islands',
'COL': 'Colombia',
'COM': 'Comoros',
'CPV': 'Cabo Verde',
'CRI': 'Costa Rica',
'CUB': 'Cuba',
'CUW': 'Curaçao',
'CXR': 'Christmas Island',
'CYM': 'Cayman Islands',
'CYP': 'Cyprus',
'CZE': 'Czech Republic',
'DEU': 'Germany',
'DJI': 'Djibouti',
'DMA': 'Dominica',
'DNK': 'Denmark',
'DOM': 'Dominican Republic',
'DZA': 'Algeria',
'ECU': 'Ecuador',
'EGY': 'Egypt',
'ERI': 'Eritrea',
'ESH': 'Western Sahara',
'ESP': 'Spain',
'EST': 'Estonia',
'ETH': 'Ethiopia',
'FIN': 'Finland',
'FJI': 'Fiji',
'FLK': 'Falkland Islands',
'FRA': 'France',
'FRO': 'Faroe Islands',
'FSM': 'Federated States of Micronesia',
'GAB': 'Gabon',
'GBR': 'United Kingdom',
'GEO': 'Georgia',
'GGY': 'Guernsey',
'GHA': 'Ghana',
'GIB': 'Gibraltar',
'GIN': 'Guinea',
'GLP': 'Guadeloupe',
'GMB': 'The Gambia',
'GNB': 'Guinea-Bissau',
'GNQ': 'Equatorial Guinea',
'GRC': 'Greece',
'GRD': 'Grenada',
'GRL': 'Greenland',
'GTM': 'Guatemala',
'GUF': 'French Guiana',
'GUM': 'Guam',
'GUY': 'Guyana',
'HKG': 'Hong Kong',
'HMD': 'Heard Island and McDonald Islands',
'HND': 'Honduras',
'HRV': 'Croatia',
'HTI': 'Haiti',
'HUN': 'Hungary',
'IDN': 'Indonesia',
'IMN': 'Isle of Man',
'IND': 'India',
'IOT': 'British Indian Ocean Territory',
'IRL': 'Ireland',
'IRN': 'Iran',
'IRQ': 'Iraq',
'ISL': 'Iceland',
'ISR': 'Israel',
'ITA': 'Italy',
'JAM': 'Jamaica',
'JEY': 'Jersey',
'JOR': 'Jordan',
'JPN': 'Japan',
'KAZ': 'Kazakhstan',
'KEN': 'Kenya',
'KGZ': 'Kyrgyzstan',
'KHM': 'Cambodia',
'KIR': 'Kiribati',
'KNA': 'Saint Kitts and Nevis',
'KOR': 'South Korea',
'KWT': 'Kuwait',
'LAO': 'Laos',
'LBN': 'Lebanon',
'LBR': 'Liberia',
'LBY': 'Libya',
'LCA': 'Saint Lucia',
'LIE': 'Liechtenstein',
'LKA': 'Sri Lanka',
'LSO': 'Lesotho',
'LTU': 'Lithuania',
'LUX': 'Luxembourg',
'LVA': 'Latvia',
'MAC': 'Macau|Macao',
'MAF': 'Collectivity of Saint Martin',
'MAR': 'Morocco',
'MCO': 'Monaco',
'MDA': 'Moldova',
'MDG': 'Madagascar',
'MDV': 'Maldives',
'MEX': 'Mexico',
'MHL': 'Marshall Islands',
'MKD': 'North Macedonia',
'MLI': 'Mali',
'MLT': 'Malta',
'MMR': 'Myanmar',
'MNE': 'Montenegro',
'MNG': 'Mongolia',
'MNP': 'Northern Mariana Islands',
'MOZ': 'Mozambique',
'MRT': 'Mauritania',
'MSR': 'Montserrat',
'MTQ': 'Martinique',
'MUS': 'Mauritius',
'MWI': 'Malawi',
'MYS': 'Malaysia',
'MYT': 'Mayotte',
'NAM': 'Namibia',
'NCL': 'New Caledonia',
'NER': 'Niger',
'NFK': 'Norfolk Island',
'NGA': 'Nigeria',
'NIC': 'Nicaragua',
'NIU': 'Niue',
'NLD': 'Netherlands',
'NOR': 'Norway',
'NPL': 'Nepal',
'NRU': 'Nauru',
'NZL': 'New Zealand',
'OMN': 'Oman',
'PAK': 'Pakistan',
'PAN': 'Panama',
'PCN': 'Pitcairn Islands',
'PER': 'Peru',
'PHL': 'Philippines',
'PLW': 'Palau',
'PNG': 'Papua New Guinea',
'POL': 'Poland',
'PRI': 'Puerto Rico',
'PRK': 'North Korea',
'PRT': 'Portugal',
'PRY': 'Paraguay',
'PSE': 'State of Palestine',
'PYF': 'French Polynesia',
'QAT': 'Qatar',
'REU': 'Réunion',
'ROU': 'Romania',
'RUS': 'Russia',
'RWA': 'Rwanda',
'SAU': 'Saudi Arabia',
'SDN': 'Sudan',
'SEN': 'Senegal',
'SGP': 'Singapore',
'SGS': 'South Georgia and the South Sandwich Islands',
'SHN': 'Saint Helena, Ascension and Tristan da Cunha',
'SJM': 'Svalbard and Jan Mayen',
'SLB': 'Solomon Islands',
'SLE': 'Sierra Leone',
'SLV': 'El Salvador',
'SMR': 'San Marino',
'SOM': 'Somalia',
'SPM': 'Saint Pierre and Miquelon',
'SRB': 'Serbia',
'SSD': 'South Sudan',
'STP': 'São Tomé and Príncipe',
'SUR': 'Suriname',
'SVK': 'Slovakia',
'SVN': 'Slovenia',
'SWE': 'Sweden',
'SWZ': 'Eswatini',
'SXM': 'Sint Maarten|',
'SYC': 'Seychelles',
'SYR': 'Syria',
'TCA': 'Turks and Caicos Islands',
'TCD': 'Chad',
'TGO': 'Togo',
'THA': 'Thailand',
'TJK': 'Tajikistan',
'TKL': 'Tokelau',
'TKM': 'Turkmenistan',
'TLS': 'East Timor',
'TON': 'Tonga',
'TTO': 'Trinidad and Tobago',
'TUN': 'Tunisia',
'TUR': 'Turkey',
'TUV': 'Tuvalu',
'TWN': 'Taiwan',
'TZA': 'Tanzania',
'UGA': 'Uganda',
'UKR': 'Ukraine',
'UMI': 'United States Minor Outlying Islands',
'URY': 'Uruguay',
'USA': 'United States',
'UZB': 'Uzbekistan',
'VAT': 'Vatican City',
'VCT': 'Saint Vincent and the Grenadines',
'VEN': 'Venezuela',
'VGB': 'British Virgin Islands',
'VIR': 'United States Virgin Islands',
'VNM': 'Vietnam',
'VUT': 'Vanuatu',
'WLF': 'Wallis and Futuna',
'WSM': 'Samoa',
'YEM': 'Yemen',
'ZAF': 'South Africa',
'ZMB': 'Zambia',
'ZWE': 'Zimbabwe',
}

articles = []

with open('../topicmodeling/ns0.3.tsv') as fin:
    tsv_reader = csv.reader(fin, delimiter='\t')
    counter = 0
    column_names = next(tsv_reader)
    for line in tsv_reader:
        counter += 1
        if counter > 10000000:
            break
        else:
            if line[0] != 'en':
                continue
            title = line[1]
            page_id = line[2]
            infobox_name, division_type, division_name = line[4].split('~~~')
            if infobox_name in settlement_countries:
                country = settlement_countries[infobox_name]
                division_granularity = 'COUNTRY OR TERRITORY'
            elif (re.match('^(country|sovereign state|sovereign\\xa0state|british overseas territory)', mwparserfromhell.parse(division_type).strip_code(), re.I)):
                division_granularity = 'COUNTRY OR TERRITORY'
                wikitext = mwparserfromhell.parse(division_name)
                wikilinks = wikitext.filter_wikilinks()
                templates = wikitext.filter_templates()
                country = None
                if len(wikilinks) == 0 and len(templates) == 0:
                    country = mwparserfromhell.parse(division_name.strip()).strip_code()
                elif len(wikilinks) == 1 and len(templates) == 0:
                    country = wikitext.strip_code()
                elif len(templates) == 1:
                    template_name = str(templates[0].name).strip().replace("_", " ")
                    if template_name.lower() in ['flag', 'flagu', 'flag icon', 'flagicon', 'flagcountry'] and len(templates[0].params) > 0:
                        country = str((templates[0].params)[0])
                        if country in template_countries:
                            country = template_countries[country]
                    elif template_name in template_countries:
                        country = template_countries[template_name]
                    elif template_name.lower() == 'nowrap':
                        country = str((templates[0].params)[0])
                        if country in template_countries:
                            country = template_countries[country]
                    else:
                        country = 'UNKNOWN: ' + division_name
                elif len(templates) > 1:
                    template_name = str(templates[0].name).strip().replace("_", " ").lower()
                    if template_name == 'nowrap':
                        template_name = str(templates[1].name).strip().replace("_", " ")
                        if template_name.lower() in ['flag', 'flagu', 'flag icon', 'flagicon', 'flagcountry'] and len(templates[1].params) > 0:
                            country = str((templates[1].params)[0])
                            if country in template_countries:
                                country = template_countries[country]
                        elif template_name in template_countries:
                            country = template_countries[template_name]
                        else:
                            country = 'UNKNOWN: ' + division_name
                    else:
                        template_name = str(templates[0].name).strip().replace("_", " ")
                        if template_name.lower() in ['flag', 'flagu', 'flag icon', 'flagicon', 'flagcountry'] and len(templates[0].params) > 0:
                            country = str((templates[0].params)[0])
                            if country in template_countries:
                                country = template_countries[country]
                        elif template_name in template_countries:
                            country = template_countries[template_name]
                        elif template_name.lower() == 'nowrap':
                            country = str((templates[0].params)[0])
                            if country in template_countries:
                                country = template_countries[country]
                        else:
                            country = 'UNKNOWN: ' + division_name
                elif len(wikilinks) == 2:
                    for link in wikilinks:
                        link_title = str(link.title)
                        if (re.match('^(Image|File):', link_title, re.I)):
                            continue
                        country = link_title
                    country = country if country else 'UNKNOWN: ' + division_name
                    # print(line)
                else:
                    country = 'UNKNOWN: ' + division_name
            elif (re.match('^State$', mwparserfromhell.parse(division_type).strip_code(), re.I)):
                division_granularity = 'State'
                if (re.match('\{\{flag\|kingdom of denmark\}\}', division_name, re.I)):
                    division_granularity = 'Country'
                    country = 'Denmark'
                elif division_name == '[[State of Palestine]]':
                    division_granularity = 'State'
                    country = 'State of Palestine'
                else:
                    country = 'UNKNOWN: ' + division_name
            else:
                division_granularity = 'UNKNOWN: ' + division_type
                country = 'UNKNOWN: ' + division_name
            country = country.replace('USA', "United States")
            country = country.replace("People's Republic of China", "China")
            articles.append([title, page_id, infobox_name, country, division_granularity])
            
            # TODO: It's mostly dealt with, but look a little closer at '^en' ns0.1.tsv | cut -f5 | sed 's/~~~/\t/g' | cut -f2,3 | grep '^State' | sort | uniq -c | sort -nr | less
            # TODO: Remove HTML comments and strip space if any further examples (historically, see Ramree Island)
            # TODO: 279039	La Zona (Santurce)	28643449	Infobox settlement	UNK UNK:[[Puerto Rico]]	UNK:[[Territories of the United States#Unincor...

    article_df = pd.DataFrame(articles, columns=['title', 'page_id_ns_0', 'infobox_name', 'country', 'division_granularity'])


In [122]:
article_df.shape

(436961, 5)

In [121]:
article_df.sample(n=60)

,title,page_id_ns_0,infobox_name,country,division_granularity
327296,Nowsara,36380379,Infobox settlement,Iran,COUNTRY OR TERRITORY
303949,Perali,32967765,Infobox settlement,India,COUNTRY OR TERRITORY
158092,Saint-Cyr-de-Favières,15809535,Infobox French commune,France,COUNTRY OR TERRITORY
326405,Posht-e Sabad,36310834,Infobox settlement,Iran,COUNTRY OR TERRITORY
154336,Saint-Marcet,15775720,Infobox French commune,France,COUNTRY OR TERRITORY
423428,Novoustinovka,60860565,Infobox settlement,Russia,COUNTRY OR TERRITORY
143757,Shaharut,15441304,Infobox Israel village,Israel,COUNTRY OR TERRITORY
285112,"Greenfields Village, New Jersey",28698864,Infobox settlement,United States,COUNTRY OR TERRITORY
157187,"Fallon, Haute-Saône",15802747,Infobox French commune,France,COUNTRY OR TERRITORY
39151,"Saijō, Hiroshima (Shōbara)",503694,Infobox city Japan,Japan,COUNTRY OR TERRITORY


In [123]:
consolidated.dtypes

page_id                             int64
page_title_x                       object
rev_id                              int64
page_title_y                       object
page_latest                       float64
is_human                          float64
has_geo                           float64
title                              object
page_id_ns_1                       object
topic                              object
topic_rating                       object
topic_first_encountered            object
topic_first_encountered_rating     object
best1                              object
best1_score                       float64
best2                              object
best2_score                       float64
best3                              object
best3_score                       float64
best4                              object
best4_score                       float64
best5                              object
best5_score                       float64
dtype: object

In [124]:
article_df.dtypes

title                   object
page_id_ns_0            object
infobox_name            object
country                 object
division_granularity    object
dtype: object

In [125]:
article_df['page_id_ns_0'] = article_df['page_id_ns_0'].astype('Float64')

In [126]:
article_df.dtypes

title                    object
page_id_ns_0            float64
infobox_name             object
country                  object
division_granularity     object
dtype: object

In [127]:
article_df

,title,page_id_ns_0,infobox_name,country,division_granularity
0,Alabama,303.0,Infobox U.S. state,United States,COUNTRY OR TERRITORY
1,Alaska,624.0,Infobox U.S. state,United States,COUNTRY OR TERRITORY
2,Alberta,717.0,Infobox province or territory of Canada,Canada,COUNTRY OR TERRITORY
3,Ankara,802.0,Infobox settlement,Turkey,COUNTRY OR TERRITORY
4,Azincourt,953.0,Infobox French commune,France,COUNTRY OR TERRITORY
5,Aalborg Municipality,1036.0,Infobox settlement,Denmark,COUNTRY OR TERRITORY
6,Aarhus,1038.0,Infobox settlement,Denmark,COUNTRY OR TERRITORY
7,Athens,1216.0,Infobox settlement,Greece,COUNTRY OR TERRITORY
8,"Ames, Iowa",1298.0,Infobox settlement,United States,COUNTRY OR TERRITORY
9,Abensberg,1305.0,Infobox German location,Germany,COUNTRY OR TERRITORY


## Now we fold in our country mappings for articles into the big dataframe we've been building up.

In [128]:
consolidated = pd.merge(consolidated, article_df, left_on='page_id', right_on='page_id_ns_0', how='left')

In [129]:
consolidated.shape

(5976305, 28)

In [130]:
consolidated.sample(n=60)

,page_id,page_title_x,rev_id,page_title_y,page_latest,is_human,has_geo,title_x,page_id_ns_1,topic,...,best3_score,best4,best4_score,best5,best5_score,title_y,page_id_ns_0,infobox_name,country,division_granularity
3783734,35187393,Matt_Rahn,921735420,Matt_Rahn,921735420.0,1.0,NaN,Matt_Rahn,35195653,WikiProject Biography,...,0.541910,History_And_Society.Education,0.079862,Culture.Entertainment,0.048476,NaN,NaN,NaN,NaN,NaN
219774,560908,TightVNC,925203602,TightVNC,925203602.0,NaN,NaN,TightVNC,1971319,WikiProject Java,...,0.088467,STEM.Mathematics,0.062476,History_And_Society.Business and economics,0.038210,NaN,NaN,NaN,NaN,NaN
245170,648678,Longues-sur-Mer,897593835,Longues-sur-Mer,897593835.0,NaN,1.0,Longues-sur-Mer,9150609,WikiProject France,...,0.054488,Geography.Maps,0.049866,Culture.Language and literature,0.038165,Longues-sur-Mer,648678.0,Infobox French commune,France,COUNTRY OR TERRITORY
5066272,50518564,Ponnukku_Thanga_Manasu,927771385,Ponnukku_Thanga_Manasu,927771385.0,NaN,NaN,Ponnukku_Thanga_Manasu,51723435,WikiProject India,...,0.044118,Assistance.Maintenance,0.019069,History_And_Society.Business and economics,0.013480,NaN,NaN,NaN,NaN,NaN
5091930,50815707,Lars_Mathias_Blank,891568564,Lars_Mathias_Blank,891568564.0,1.0,NaN,Lars_Mathias_Blank,50888284,WikiProject Biography,...,0.243358,STEM.Biology,0.179497,History_And_Society.History and society,0.083978,NaN,NaN,NaN,NaN,NaN
3169106,27762007,Bursa_lucaensis,876749591,Bursa_lucaensis,876749591.0,NaN,NaN,Bursa_lucaensis,27762154,WikiProject Gastropods,...,0.067076,Assistance.Maintenance,0.040874,Culture.Language and literature,0.038054,NaN,NaN,NaN,NaN,NaN
3307801,29418665,Workforce_development,897442241,Workforce_development,897442241.0,NaN,NaN,Workforce_development,29418699,WikiProject Business,...,0.112912,History_And_Society.Education,0.096104,STEM.Engineering,0.095866,NaN,NaN,NaN,NaN,NaN
4652681,44712603,DXSG,842287405,DXSG,842287405.0,NaN,NaN,DXSG,44967294,WikiProject Radio Stations,...,0.041813,Culture.Media,0.037263,History_And_Society.Business and economics,0.036855,NaN,NaN,NaN,NaN,NaN
3761393,34877755,Maalai_Pozhudhin_Mayakathilaey,871111173,Maalai_Pozhudhin_Mayakathilaey,871111173.0,NaN,NaN,Maalai_Pozhudhin_Mayakathilaey,34979859,WikiProject India,...,0.118597,Geography.Europe,0.068323,Culture.Internet culture,0.035363,NaN,NaN,NaN,NaN,NaN
2169572,17137106,WUCL_(FM),796618282,WUCL_(FM),796618282.0,NaN,1.0,WUCL_(FM),20612473,WikiProject United States,...,0.045649,Geography.Europe,0.021834,History_And_Society.Business and economics,0.018824,NaN,NaN,NaN,NaN,NaN


In [131]:
countries = [country.replace(' ', '_') for country in template_countries.values()]
    

countries_df = pd.DataFrame(countries, columns=['title'])

In [132]:
countries_df

,title
0,Iran
1,Andorra
2,Rwanda
3,Collectivity_of_Saint_Martin
4,Heard_Island_and_McDonald_Islands
5,Malta
6,Bahrain
7,Barbados
8,Aruba
9,Wallis_and_Futuna


## As it turns out some of the country name assignments in templates and "canonical" country name lists are a little messy and the earlier script output that was parsing the templates needs a little bit more treatment.

In [133]:
country_linkage = pd.merge(countries_df, consolidated, left_on='title', right_on='page_title_x', how='left')

In [134]:
country_linkage[country_linkage['page_title_x'].isnull()]

,title,page_id,page_title_x,rev_id,page_title_y,page_latest,is_human,has_geo,title_x,page_id_ns_1,...,best3_score,best4,best4_score,best5,best5_score,title_y,page_id_ns_0,infobox_name,country,division_granularity
21,Macau|Macao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,Cabo_Verde,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,Sint_Maarten|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
article_df[article_df['country'] == 'Sint Maarten']

,title,page_id_ns_0,infobox_name,country,division_granularity


In [136]:
article_df[article_df['country'] == 'Sint Maarten|']

,title,page_id_ns_0,infobox_name,country,division_granularity


In [137]:
article_df[article_df['country'] == 'Saint Martin']

,title,page_id_ns_0,infobox_name,country,division_granularity


In [138]:
article_df[article_df['country'] == 'Macau|Macao']

,title,page_id_ns_0,infobox_name,country,division_granularity


In [139]:
article_df[article_df['country'] == 'Macao']

,title,page_id_ns_0,infobox_name,country,division_granularity


In [140]:
article_df[article_df['country'] == 'Macau']

,title,page_id_ns_0,infobox_name,country,division_granularity
37049,Taipa,361320.0,Infobox settlement,Macau,COUNTRY OR TERRITORY
37050,Coloane,361321.0,Infobox settlement,Macau,COUNTRY OR TERRITORY
37057,"Nossa Senhora de Fátima, Macau",362264.0,Infobox settlement,Macau,COUNTRY OR TERRITORY
37058,"Santo António, Macau",362265.0,Infobox settlement,Macau,COUNTRY OR TERRITORY
37059,São Lázaro,362266.0,Infobox settlement,Macau,COUNTRY OR TERRITORY
37060,"Sé, Macau",362273.0,Infobox settlement,Macau,COUNTRY OR TERRITORY
37061,"São Lourenço, Macau",362275.0,Infobox settlement,Macau,COUNTRY OR TERRITORY
56775,Cotai,2446377.0,Infobox settlement,Macau,COUNTRY OR TERRITORY


In [141]:
article_df[article_df['country'] == 'Cabo Verde']

,title,page_id_ns_0,infobox_name,country,division_granularity


In [142]:
article_df[article_df['country'] == 'Cape Verde']

,title,page_id_ns_0,infobox_name,country,division_granularity
1251,Mindelo,75327.0,Infobox Cape Verde settlement,Cape Verde,COUNTRY OR TERRITORY
47008,"São Domingos, Cape Verde (municipality)",1259711.0,Infobox Cape Verde settlement,Cape Verde,COUNTRY OR TERRITORY
47169,Chã das Caldeiras,1278581.0,Infobox Cape Verde settlement,Cape Verde,COUNTRY OR TERRITORY
49169,Cidade Velha,1534217.0,Infobox Cape Verde settlement,Cape Verde,COUNTRY OR TERRITORY
49796,"São Filipe, Cape Verde (municipality)",1632103.0,Infobox Cape Verde settlement,Cape Verde,COUNTRY OR TERRITORY
49818,"São Filipe, Cape Verde",1634848.0,Infobox Cape Verde settlement,Cape Verde,COUNTRY OR TERRITORY
58391,"Porto Novo, Cape Verde (municipality)",2685773.0,Infobox Cape Verde settlement,Cape Verde,COUNTRY OR TERRITORY
58422,"Ribeira Grande, Cape Verde (municipality)",2692031.0,Infobox Cape Verde settlement,Cape Verde,COUNTRY OR TERRITORY
58461,"Paul, Cape Verde",2698414.0,Infobox Cape Verde settlement,Cape Verde,COUNTRY OR TERRITORY
58630,"Tarrafal, Cape Verde (municipality)",2726861.0,Infobox Cape Verde settlement,Cape Verde,COUNTRY OR TERRITORY


In [143]:
countries = [country.replace(' ', '_') for country in template_countries.values()] + ['Cape_Verde', 'Macau', 'Sint_Maarten']
countries_df = pd.DataFrame(countries, columns=['title'])
country_linkage = pd.merge(countries_df, consolidated, left_on='title', right_on='page_title_x', how='left')
country_linkage[country_linkage['page_title_x'].isnull()]

,title,page_id,page_title_x,rev_id,page_title_y,page_latest,is_human,has_geo,title_x,page_id_ns_1,...,best3_score,best4,best4_score,best5,best5_score,title_y,page_id_ns_0,infobox_name,country,division_granularity
21,Macau|Macao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,Cabo_Verde,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,Sint_Maarten|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [144]:
country_linkage[country_linkage['page_title_x'] == 'Macau']

,title,page_id,page_title_x,rev_id,page_title_y,page_latest,is_human,has_geo,title_x,page_id_ns_1,...,best3_score,best4,best4_score,best5,best5_score,title_y,page_id_ns_0,infobox_name,country,division_granularity
268,Macau,19068.0,Macau,928671799.0,Macau,930301444.0,NaN,1.0,Macau,233392,...,0.110359,Geography.Europe,0.106146,Assistance.Maintenance,0.103512,NaN,NaN,NaN,NaN,NaN


In [145]:
country_linkage[country_linkage['page_title_x'] == 'Cape_Verde']

,title,page_id,page_title_x,rev_id,page_title_y,page_latest,is_human,has_geo,title_x,page_id_ns_1,...,best3_score,best4,best4_score,best5,best5_score,title_y,page_id_ns_0,infobox_name,country,division_granularity
267,Cape_Verde,18962637.0,Cape_Verde,928123381.0,Cape_Verde,930317901.0,NaN,1.0,Cape_Verde,18953159,...,0.080084,History_And_Society.History and society,0.078029,Culture.Language and literature,0.053505,NaN,NaN,NaN,NaN,NaN


In [146]:
country_linkage[country_linkage['page_title_x'] == 'Sint_Maarten']

,title,page_id,page_title_x,rev_id,page_title_y,page_latest,is_human,has_geo,title_x,page_id_ns_1,...,best3_score,best4,best4_score,best5,best5_score,title_y,page_id_ns_0,infobox_name,country,division_granularity
269,Sint_Maarten,9529005.0,Sint_Maarten,927675171.0,Sint_Maarten,929957030.0,NaN,1.0,Sint_Maarten,11433674,...,0.15257,History_And_Society.History and society,0.135698,Geography.Oceania,0.063385,NaN,NaN,NaN,NaN,NaN


In [147]:
countries_df = pd.DataFrame(countries, columns=['country_direct'])

In [148]:
countries_df

,country_direct
0,Iran
1,Andorra
2,Rwanda
3,Collectivity_of_Saint_Martin
4,Heard_Island_and_McDonald_Islands
5,Malta
6,Bahrain
7,Barbados
8,Aruba
9,Wallis_and_Futuna


In [150]:
countries_df.duplicated()

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
       ...  
240    False
241     True
242    False
243    False
244    False
245    False
246    False
247     True
248    False
249    False
250    False
251    False
252    False
253    False
254     True
255    False
256    False
257    False
258    False
259    False
260     True
261     True
262    False
263    False
264    False
265    False
266    False
267    False
268    False
269    False
dtype: bool

In [151]:
countries_df.shape

(270, 1)

In [152]:
countries_df.drop_duplicates(inplace=True)

In [153]:
countries_df.shape

(253, 1)

In [154]:
countries_df[countries_df['country_direct'] == 'United_States']

,country_direct
42,United_States


## Okay, we take our fixes and fold those in for a higher confidence string literal for the country name.

In [155]:
consolidated = pd.merge(consolidated, countries_df, left_on='page_title_x', right_on='country_direct', how='left')

In [158]:
consolidated.shape

(5976305, 29)

In [157]:
pd.set_option('display.max_columns', 50)

In [159]:
consolidated[consolidated['page_title_x'] == 'United_States']

,page_id,page_title_x,rev_id,page_title_y,page_latest,is_human,has_geo,title_x,page_id_ns_1,topic,topic_rating,topic_first_encountered,topic_first_encountered_rating,best1,best1_score,best2,best2_score,best3,best3_score,best4,best4_score,best5,best5_score,title_y,page_id_ns_0,infobox_name,country,division_granularity,country_direct
782970,3434750,United_States,928636548,United_States,930315073.0,NaN,1.0,United_States,3433959,WikiProject United States,top,WikiProject United States,top,Geography.Countries,0.615518,History_And_Society.History and society,0.424982,Culture.Language and literature,0.419514,History_And_Society.Politics and government,0.34759,Assistance.Maintenance,0.174983,NaN,NaN,NaN,NaN,NaN,United_States


In [163]:
consolidated.sample(n=20)

,page_id,page_title_x,rev_id,page_title_y,page_latest,is_human,has_geo,title_x,page_id_ns_1,topic,topic_rating,topic_first_encountered,topic_first_encountered_rating,best1,best1_score,best2,best2_score,best3,best3_score,best4,best4_score,best5,best5_score,title_y,page_id_ns_0,infobox_name,country,division_granularity,country_direct
2437592,19988623,Ranking_(information_retrieval),925335191,Ranking_(information_retrieval),925335191.0,NaN,NaN,Ranking_(information_retrieval),28033608,WikiProject Computer science,unknown_importance,WikiProject Computer science,unknown_importance,Geography.Maps,0.330318,STEM.Technology,0.326651,STEM.Science,0.147641,Assistance.Maintenance,0.085214,History_And_Society.Business and economics,0.076036,NaN,NaN,NaN,NaN,NaN,NaN
3335098,29722107,Verte_Island,869634768,Verte_Island,869634768.0,NaN,1.0,Verte_Island,29742305,WikiProject Overseas France,low,WikiProject Overseas France,low,Geography.Countries,0.609765,Geography.Europe,0.223324,Geography.Bodies of water,0.143498,Geography.Landforms,0.122168,Geography.Oceania,0.095317,NaN,NaN,NaN,NaN,NaN,NaN
4764518,46536610,Jim_May_(Australian_footballer),909704840,Jim_May_(Australian_footballer),909704840.0,1.0,NaN,Jim_May_(Australian_footballer),46536619,WikiProject Australia,low,WikiProject Biography,unknown_importance,Geography.Oceania,0.995870,Culture.Language and literature,0.899733,Culture.Sports,0.068065,Geography.Countries,0.044683,Assistance.Maintenance,0.026056,NaN,NaN,NaN,NaN,NaN,NaN
1354557,8223091,1969_White_Paper,923852046,1969_White_Paper,929179415.0,NaN,NaN,1969_White_Paper,29093660,WikiProject Canada,high,WikiProject Canada,high,History_And_Society.Politics and government,0.725231,Geography.Countries,0.660709,History_And_Society.History and society,0.496280,Geography.Oceania,0.304602,Culture.Language and literature,0.121841,NaN,NaN,NaN,NaN,NaN,NaN
3918653,36623925,Abdullah_Seehan,919129836,Abdullah_Seehan,919129836.0,1.0,NaN,Abdullah_Seehan,36870226,WikiProject Football,mid,WikiProject Biography,unknown_importance,Culture.Sports,0.983342,Culture.Language and literature,0.900528,Geography.Countries,0.756524,Geography.Europe,0.046660,Assistance.Maintenance,0.019369,NaN,NaN,NaN,NaN,NaN,NaN
1976610,14925070,Gmina_Borzechów,921450996,Gmina_Borzechów,921450996.0,NaN,1.0,Gmina_Borzechów,19184162,WikiProject Poland,low,WikiProject Poland,low,Geography.Europe,0.814821,Geography.Countries,0.075451,Geography.Maps,0.053393,Geography.Cities,0.037930,Assistance.Maintenance,0.014778,Gmina Borzechów,14925070.0,Infobox settlement,Poland,COUNTRY OR TERRITORY,NaN
1768948,12730274,Joint_Control_Commission_for_Georgian–Ossetian...,919121428,Joint_Control_Commission_for_Georgian–Ossetian...,919121428.0,NaN,NaN,Joint_Control_Commission_for_Georgian–Ossetian...,12747099,WikiProject Ossetia,mid,WikiProject Military history,unknown_importance,Geography.Europe,0.563703,History_And_Society.Military and warfare,0.355159,Geography.Countries,0.264883,History_And_Society.Politics and government,0.134983,Culture.Language and literature,0.076157,NaN,NaN,NaN,NaN,NaN,NaN
3919292,36628739,Red_discography,928300511,Red_discography,928300511.0,NaN,NaN,Red_discography,44771328,WikiProject Discographies,unknown_importance,WikiProject Discographies,unknown_importance,Culture.Performing arts,0.560410,Culture.Philosophy and religion,0.132511,Geography.Europe,0.127154,Geography.Countries,0.075803,Assistance.Maintenance,0.070255,NaN,NaN,NaN,NaN,NaN,NaN
2624768,22000665,Nowy_Przybysław,772500241,Nowy_Przybysław,772500241.0,NaN,1.0,Nowy_Przybysław,22922848,WikiProject Poland,mid,WikiProject Poland,mid,Geography.Europe,0.891542,Geography.Countries,0.205905,Geography.Cities,0.127166,History_And_Society.History and society,0.016740,Assistance.Maintenance,0.016582,Nowy Przybysław,22000665.0,Infobox settlement,Poland,COUNTRY OR TERRITORY,NaN
5123260,51195225,Mianwal_Araian,848390540,Mianwal_Araian,848390540.0,NaN,1.0,Mianwal_Araian,51195605,WikiProject India,low,WikiProject India,low,Geography.C

## In the course of data cleaning, there were some standout issues with extraneous characters in some of the fields, so the following code deals with that.

In [165]:
consolidated[consolidated['page_title_x'] == 'Lift_(force)']

,page_id,page_title_x,rev_id,page_title_y,page_latest,is_human,has_geo,title_x,page_id_ns_1,topic,topic_rating,topic_first_encountered,topic_first_encountered_rating,best1,best1_score,best2,best2_score,best3,best3_score,best4,best4_score,best5,best5_score,title_y,page_id_ns_0,infobox_name,country,division_granularity,country_direct
8749,18009,Lift_(force),928107641,Lift_(force),930149311.0,NaN,NaN,Lift_(force),21467140,WikiProject Physics,mid,WikiProject Aviation,high\n,STEM.Physics,0.81973,STEM.Engineering,0.29792,History_And_Society.Transportation,0.107202,STEM.Technology,0.09679,History_And_Society.History and society,0.041129,NaN,NaN,NaN,NaN,NaN,NaN


In [166]:
consolidated[consolidated['page_title_x'] == 'Epirus']

,page_id,page_title_x,rev_id,page_title_y,page_latest,is_human,has_geo,title_x,page_id_ns_1,topic,topic_rating,topic_first_encountered,topic_first_encountered_rating,best1,best1_score,best2,best2_score,best3,best3_score,best4,best4_score,best5,best5_score,title_y,page_id_ns_0,infobox_name,country,division_granularity,country_direct
19376,38260,Epirus,927398406,Epirus,927398406.0,NaN,NaN,Epirus,189127,WikiProject Albania,high,WikiProject Albania,high,Geography.Europe,0.867,History_And_Society.History and society,0.782515,Geography.Countries,0.345711,History_And_Society.Military and warfare,0.282419,Geography.Maps,0.167334,Epirus,38260.0,Infobox settlement,UNKNOWN: Divided between [[Greece]] and [[Alba...,UNKNOWN: Present status,NaN


In [167]:
consolidated.dtypes

page_id                             int64
page_title_x                       object
rev_id                              int64
page_title_y                       object
page_latest                       float64
is_human                          float64
has_geo                           float64
title_x                            object
page_id_ns_1                       object
topic                              object
topic_rating                       object
topic_first_encountered            object
topic_first_encountered_rating     object
best1                              object
best1_score                       float64
best2                              object
best2_score                       float64
best3                              object
best3_score                       float64
best4                              object
best4_score                       float64
best5                              object
best5_score                       float64
title_y                           

In [168]:
consolidated['country'] = consolidated['country'].apply(lambda x: str(x).replace("\n", " "))

In [169]:
print(consolidated[consolidated['page_title_x'] == 'Epirus']['country'])

19376    UNKNOWN: Divided between [[Greece]] and [[Alba...
Name: country, dtype: object


In [170]:
consolidated['topic_first_encountered_rating'] = consolidated['topic_first_encountered_rating'].apply(lambda x: str(x).replace("\n", " "))

In [171]:
consolidated[consolidated['page_title_x'] == 'Lift_(force)']

,page_id,page_title_x,rev_id,page_title_y,page_latest,is_human,has_geo,title_x,page_id_ns_1,topic,topic_rating,topic_first_encountered,topic_first_encountered_rating,best1,best1_score,best2,best2_score,best3,best3_score,best4,best4_score,best5,best5_score,title_y,page_id_ns_0,infobox_name,country,division_granularity,country_direct
8749,18009,Lift_(force),928107641,Lift_(force),930149311.0,NaN,NaN,Lift_(force),21467140,WikiProject Physics,mid,WikiProject Aviation,high,STEM.Physics,0.81973,STEM.Engineering,0.29792,History_And_Society.Transportation,0.107202,STEM.Technology,0.09679,History_And_Society.History and society,0.041129,NaN,NaN,NaN,nan,NaN,NaN


In [172]:
print(consolidated[consolidated['page_title_x'] == 'Lift_(force)']['topic_first_encountered_rating'])

8749    high  
Name: topic_first_encountered_rating, dtype: object


In [173]:
consolidated['topic_first_encountered_rating'] = consolidated['topic_first_encountered_rating'].apply(lambda x: str(x).replace(" ", ""))

In [174]:
print(consolidated[consolidated['page_title_x'] == 'Lift_(force)']['topic_first_encountered_rating'])

8749    high
Name: topic_first_encountered_rating, dtype: object


In [175]:
consolidated['topic_rating'] = consolidated['topic_rating'].apply(lambda x: str(x).replace("\n", ""))

In [176]:
consolidated['division_granularity'] = consolidated['division_granularity'].apply(lambda x: str(x).replace("\n", " "))

In [177]:
consolidated[consolidated['page_title_x'] == 'Dera_Gopipur']

,page_id,page_title_x,rev_id,page_title_y,page_latest,is_human,has_geo,title_x,page_id_ns_1,topic,topic_rating,topic_first_encountered,topic_first_encountered_rating,best1,best1_score,best2,best2_score,best3,best3_score,best4,best4_score,best5,best5_score,title_y,page_id_ns_0,infobox_name,country,division_granularity,country_direct
1082904,5725600,Dera_Gopipur,919708207,Dera_Gopipur,919708207.0,NaN,1.0,Dera_Gopipur,10112221,WikiProject India,low,WikiProject India,low,Geography.Countries,0.945511,Geography.Europe,0.104835,History_And_Society.History and society,0.060465,Assistance.Maintenance,0.053204,Culture.Plastic arts,0.037093,Dera Gopipur,5725600.0,Infobox settlement,UNKNOWN: Dehra Gopipur {{flag|India}},UNKNOWN: Administrative Country,NaN


In [178]:
print(consolidated[consolidated['page_title_x'] == 'East_North_Central_states']['country'])

305934    UNKNOWN: {{Ubl|list_style=line-height: inherit...
Name: country, dtype: object


In [179]:
consolidated[consolidated['page_title_x'] == 'Bampton_Castle,_Oxfordshire']

,page_id,page_title_x,rev_id,page_title_y,page_latest,is_human,has_geo,title_x,page_id_ns_1,topic,topic_rating,topic_first_encountered,topic_first_encountered_rating,best1,best1_score,best2,best2_score,best3,best3_score,best4,best4_score,best5,best5_score,title_y,page_id_ns_0,infobox_name,country,division_granularity,country_direct
1369392,8382977,"Bampton_Castle,_Oxfordshire",916387109,"Bampton_Castle,_Oxfordshire",916387109.0,NaN,1.0,"Bampton_Castle,_Oxfordshire",14590733,WikiProject England,low,WikiProject Military history\n,unknown_importance,Geography.Europe,0.9193,Culture.Language and literature,0.24074,Culture.Plastic arts,0.147655,History_And_Society.History and society,0.146175,Geography.Maps,0.131735,NaN,NaN,NaN,nan,nan,NaN


In [180]:
consolidated[consolidated['page_title_x'] == 'Dextra_Manufacturing']

,page_id,page_title_x,rev_id,page_title_y,page_latest,is_human,has_geo,title_x,page_id_ns_1,topic,topic_rating,topic_first_encountered,topic_first_encountered_rating,best1,best1_score,best2,best2_score,best3,best3_score,best4,best4_score,best5,best5_score,title_y,page_id_ns_0,infobox_name,country,division_granularity,country_direct
3451503,31243807,Dextra_Manufacturing,918291158,Dextra_Manufacturing,918291158.0,NaN,NaN,Dextra_Manufacturing,41307718,WikiProject Thailand,low,WikiProject \tCompanies,unknown_importance,STEM.Engineering,0.761485,STEM.Technology,0.449747,History_And_Society.Business and economics,0.303934,Geography.Countries,0.151824,Assistance.Maintenance,0.062691,NaN,NaN,NaN,nan,nan,NaN


In [181]:
consolidated['topic_first_encountered'] = consolidated['topic_first_encountered'].apply(lambda x: str(x).replace("\n", ""))

In [182]:
consolidated['topic_first_encountered'] = consolidated['topic_first_encountered'].apply(lambda x: str(x).replace("\t", ""))

In [183]:
consolidated.groupby(['topic_rating']).size()

topic_rating
high                   143904
low                   3209588
mid                    633256
nan                    182178
top                     35216
unknown_importance    1772163
dtype: int64

In [184]:
consolidated.groupby(['topic_first_encountered_rating']).size().reset_index(name='counts').sort_values(by=['counts'])

,topic_first_encountered_rating,counts
0,',1
103,low}mtcities=yes,1
101,lowy,1
99,lowv,1
98,lowuganda-importance=mid,1
96,lowt,1
95,lowst,1
94,lowsingingsword,1
93,lowsfba=yes,1
91,lowr,1


In [185]:
consolidated

,page_id,page_title_x,rev_id,page_title_y,page_latest,is_human,has_geo,title_x,page_id_ns_1,topic,topic_rating,topic_first_encountered,topic_first_encountered_rating,best1,best1_score,best2,best2_score,best3,best3_score,best4,best4_score,best5,best5_score,title_y,page_id_ns_0,infobox_name,country,division_granularity,country_direct
0,12,Anarchism,928659071,Anarchism,930305821.0,NaN,NaN,Anarchism,416541,WikiProject Alternative Views,top,WikiProject Alternative Views,top,Culture.Language and literature,0.573744,History_And_Society.History and society,0.564499,Culture.Philosophy and religion,0.442044,Geography.Europe,0.337839,History_And_Society.Politics and government,0.305153,NaN,NaN,NaN,nan,nan,NaN
1,25,Autism,928602598,Autism,928602598.0,NaN,NaN,Autism,44741,WikiProject Autism,top,WikiProject Medicine,high,STEM.Medicine,0.990639,History_And_Society.History and society,0.451306,STEM.Biology,0.215708,Culture.Language and literature,0.140741,Geography.Countries,0.062510,NaN,NaN,NaN,nan,nan,NaN
2,39,Albedo,926891749,Albedo,926891749.0,NaN,NaN,Albedo,47435,WikiProject Astronomy,high,WikiProject Physics,low,STEM.Space,0.690529,STEM.Meteorology,0.677404,STEM.Physics,0.331766,STEM.Geosciences,0.178165,History_And_Society.History and society,0.103957,NaN,NaN,NaN,nan,nan,NaN
3,290,A,927107123,A,927107123.0,NaN,NaN,A,2554,WikiProject Writing systems,mid,WikiProject Writing systems,mid,Culture.Language and literature,0.336972,Geography.Europe,0.201948,History_And_Society.History and society,0.155678,Geography.Countries,0.127005,Assistance.Maintenance,0.079002,NaN,NaN,NaN,nan,nan,NaN
4,303,Alabama,928520502,Alabama,930263332.0,NaN,1.0,Alabama,1284,WikiProject Alabama,top,WikiProject Alabama,top,Geography.Countries,0.809819,History_And_Society.History and society,0.171643,History_And_Society.Politics and government,0.133188,Assistance.Maintenance,0.064497,Culture.Language and literature,0.064399,Alabama,303.0,Infobox U.S. state,United States,COUNTRY OR TERRITORY,NaN
5,305,Achilles,928351599,Achilles,928951332.0,NaN,NaN,Achilles,543840,WikiProject Greece,high,WikiProject Biography,unknown_importance,History_And_Society.History and society,0.690594,Culture.Language and literature,0.469709,Geography.Europe,0.451748,Culture.Philosophy and religion,0.342202,Culture.Visual arts,0.148593,NaN,NaN,NaN,nan,nan,NaN
6,307,Abraham_Lincoln,928533917,Abraham_Lincoln,930094843.0,1.0,NaN,Abraham_Lincoln,2301,WikiProject Illinois,top,WikiProject Biography,unknown_importance,Geography.Countries,0.789089,Culture.Language and literature,0.762656,History_And_Society.History and society,0.322005,History_And_Society.Military and warfare,0.287212,History_And_Society.Politics and government,0.184107,NaN,NaN,NaN,nan,nan,NaN
7,308,Aristotle,928197558,Aristotle,929227456.0,1.0,NaN,Aristotle,18539950,WikiProject Philosophy,top,WikiProject Biography,unknown_importance,Culture.Language and literature,0.743942,Culture.Philosophy and religion,0.603628,History_And_Society.History and society,0.595194,Geography.Europe,0.177480,STEM.Science,0.111327,NaN,NaN,NaN,nan,nan,NaN
8,309,An_American_in_Paris,926138379,An_American_in_Paris,930213356.0,NaN,NaN,An_American_in_Paris,1229991,WikiProject Classical music,unknown_importance,WikiProject Classical music,unknown_importance,Culture.Performing arts,0.969374,Culture.Language and literature,0.500320,Geography.Countries,0.188465,Geography.Europe,0.136918,History_And_Society.History and society,0.065426,NaN,NaN,NaN,nan,nan,NaN
9,316,Academy_Award_for_Best_Production_Design,923578186,Academy_Award_for_Best_Production_Design,923578186.0,NaN,NaN,Academy_Award_for_Best_Production_Design,4292425,WikiProject Awards,mid,WikiProject Film,unknown_importance,Culture.Entertainment,0.937438,Assistance.Maintenance,0.177690,Culture.Visual arts,0.148124,Culture.Language and literature,0.106589,Culture.Performing arts,0.093777,NaN,NaN,NaN,nan,nan,NaN


In [187]:
country_projects = ['WikiProject ' + v for v in list(set(template_countries.values()))]
country_projects

['WikiProject French Southern and Antarctic Lands',
 'WikiProject Brunei',
 'WikiProject Philippines',
 'WikiProject Malaysia',
 'WikiProject Marshall Islands',
 'WikiProject Eswatini',
 'WikiProject Bouvet Island',
 'WikiProject Belarus',
 'WikiProject Ireland',
 'WikiProject Latvia',
 'WikiProject Ecuador',
 'WikiProject Barbados',
 'WikiProject Sudan',
 'WikiProject New Zealand',
 'WikiProject Solomon Islands',
 'WikiProject Jersey',
 'WikiProject Collectivity of Saint Martin',
 'WikiProject Libya',
 'WikiProject North Korea',
 'WikiProject Argentina',
 'WikiProject Vanuatu',
 'WikiProject Curaçao',
 'WikiProject Chile',
 'WikiProject Tanzania',
 'WikiProject Faroe Islands',
 'WikiProject Mexico',
 'WikiProject Iceland',
 'WikiProject Ghana',
 'WikiProject Bermuda',
 'WikiProject Yemen',
 'WikiProject France',
 'WikiProject Dominican Republic',
 'WikiProject Kyrgyzstan',
 'WikiProject Papua New Guinea',
 'WikiProject Spain',
 'WikiProject Oman',
 'WikiProject Kiribati',
 'WikiProjec

## Yay, things are pretty cleaned up. Let's get this big dataframe into a TSV for further processing.

In [186]:
consolidated.to_csv(path_or_buf='topic_enwiki_revisions_201912_mediawiki_page_dump_enriched_20191201_through_20191219.tsv', sep="\t")

In [188]:
%%bash
gzip topic_enwiki_revisions_201912_mediawiki_page_dump_enriched_20191201_through_20191219.tsv

## I download that file to my local machine and run a Python script against it

See that script at https://github.com/dr0ptp4kt/dr0ptp4kt.github.io/blob/master/parse-8.py

You'll see there that it opens the mid level category file, which I touch up lightly to use the STEM.Science and STEM.Meteorology topic strings to match drafttopic outputs. See that at https://github.com/dr0ptp4kt/dr0ptp4kt.github.io/blob/master/outmid.20191219.json_2019-12-19.refined. The fetching was part of the drafttopic build scripts, which I've modded to use MCR slots in the Action API and relaxed some regexes (pull request submitted at https://github.com/wikimedia/drafttopic/pull/40).

That script lets one generate an HTML file and a big TSV that can then later be joined to other data.



You can see an example of a report joining the big TSV at https://github.com/dr0ptp4kt/dr0ptp4kt.github.io/blob/master/TopicPageviewsNov2019.ipynb (sorry, variable names in that file refer to August and September, but it's definitely for October and November as you'll see in the queries). This was a copy-paste and update from https://nbviewer.jupyter.org/github/conniecc1/topics-modeling/blob/master/TopicPageviewsSept2019.ipynb.